### Extract meetup data using meetup API 

Extract the meetup data of categories, groups, members, events and rsvps to build our recommendation engine. 
[Meetup api](https://www.meetup.com/meetup_api/)

Note: These scripts will work only with Python 2.7 and not with Python 3.0+ 

In [2]:
import requests
import json
import time
import codecs
import sys
import time
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import json
import os
import datetime
import re
import random, copy
import os

#Add your key here
api_key = ''

UTF8Writer = codecs.getwriter('utf8')
sys.stdout = UTF8Writer(sys.stdout)
url_string = "http://api.meetup.com//"

import meetup.api
client = meetup.api.Client(api_key,overlimit_wait=True)
a = client.GetCategories()

29/30 (10 seconds remaining)


In [ ]:
## Fetch the category,cities,topic_categories information using meetup.api package

## Categories
categories_df = pd.read_json(json.dumps(a.results))
categories_df.to_csv('categories_new.csv', index = False)

## Cities
cities = client.GetCities(country='US', state='TX')
cities_df = pd.read_json(json.dumps(cities.results))
cities_df.to_csv('cities_new.csv',index=False)

## Topics
topic_categories = client.GetTopicCategories()
topic_catg_df = pd.read_json(json.dumps(topic_categories.results))
topic_catg_df.to_csv('topic_categories_new.csv',index=False)

In [ ]:
## Read Categories File
df = pd.read_csv('categories_new.csv')
catg_id = df.id

For the purposes of our analysis, we extracted all the Meetup groups from Austin, Texas

In [ ]:
## Function to fetch groups
def getgroups(catg_id,off_val):
    group = client.GetGroups(category_id = catg_id,offset = off_val,location = 'Austin')
    result = group.results
    RES = json_normalize(result)
    return RES

In [ ]:
## Fetch unique groups 
cwd = os.getcwd()
group_id_extract = pd.DataFrame([])
dat_list = []
for i in catg_id:
    print (i)
    try:
        data = pd.read_csv(cwd+'\\groups_'+str(i)+'.csv',usecols=[13,4], index_col = False, header=0)
        dat_list.append(data)
    except:
        continue
group_id_extract = pd.concat(dat_list)
group_id_new = group_id_extract[group_id_extract['country']=='US']['id']
group_id_US = group_id_new.drop_duplicates().reset_index(drop=True)
unique_groups = np.array(group_id_US)

In [ ]:
## Create groups files for every category
for i in range(len(catg_id)):
    groups = pd.DataFrame([])
    for j in range(10000):
        try:
            result = getgroups(catg_id[i],j)
        except:
            continue
        if len(result) != 0:
            groups = groups.append(result)
        else:
            break
    groups.to_csv('groups_'+ str(catg_id[i]) +'.csv',index=False,encoding='utf-8')

## Concatenate all the group files that were fetched indiviually

In [ ]:
## Call the Meetup API
def get_results(string, params):
    request = requests.get(string,params=params)
    return request.json()

In [ ]:
## Function to fetch members data
def get_members( group_id, index,num_groups ):
    result =[]
    prev=[]
    for offset in range(10000):
        count = 0
        while True:
            params = {"sign":"true", "key":api_key, "page":200, "offset":offset, "group_id":group_id}
            try:
                time.sleep(0.1)
                temp = get_results(url_string + "2/members", params)
            except (ValueError, ChunkedEncodingError):
                continue
            break
        if len(temp['results']) != 0:
            if prev == temp['results']:
                value = pd.DataFrame(pd.io.json.json_normalize(result))
                value['group_id']=[group_id] *len(value)
                
                value.to_csv("member_"+str(group_id)+'.csv', encoding = 'utf-8',sep=str(","))
                return None
            result = result + temp['results']
            sys.stdout.write('\r' + "Processed: " + str(len(result)) + " events for group number " + str(group_id) +\
                             ". Progress: "+ str(index) +'/'+str(num_groups))
            sys.stdout.flush()
            prev = copy.deepcopy(temp['results'])
        else:
            break
    sys.stdout.write('\r' + " "*150 + '\r')
    value = pd.DataFrame(pd.io.json.json_normalize(result))
    value['group_id']=[group_id] *len(value)
    value.to_csv("member_"+str(group_id)+'.csv', encoding = 'utf-8',sep=str(","))

In [ ]:
## Fetch members and extract to csv
num_groups = len(unique_groups)
for index, group_id in enumerate(unique_groups):
    (get_members(group_id,index, num_groups))

In [ ]:
## Function to fetch events
def getevents(group_id,off_val):
    event = client.GetEvents(group_id = group_id, offset = off_val)
    result = event.results
    RES = json_normalize(result)
    return RES

In [ ]:
## Fetch events and extract to csv
events = pd.DataFrame([])
for i in range(len(unique_groups)):
    for j in range(10000):
        result = getevents(unique_groups.id[i], j)
        if len(result) != 0:
            events = events.append(result)
        else:
            break
events.to_csv('events.csv',index=False,encoding='utf-8')

In [ ]:
## Read the Austin groups into a csv file
austin_groups = pd.read_csv("groups_austin.csv")
austin_groups_ids = np.array((austin_groups.id))
austin_groups_ids=list(set(list(austin_groups_ids)))

In [ ]:
## Fetch events and extract to csv
events = pd.DataFrame([])
for i in range(len(austin_groups_ids)):
    for j in range(10000):
        result = getevents(temp_group_ids[i], j)
        if len(result) != 0:
            events = events.append(result)
        else:
            break
events.to_csv('events_new_100_rest.csv',index=False,encoding='utf-8')

In [2]:
## Store and reading files before fetching the RSVP data
events_new = pd.read_csv("events_new.csv")
events_new_100_rest = pd.read_csv('events_new_100_rest.csv')
events_all_new = pd.concat([events_new,events_new_100_rest])
events_all_new.to_csv('events_all_new.csv',index=False,encoding='utf-8')
events_all_new = pd.read_csv("events_all_new.csv")
event_ids = list(events_all_new.id)
#austin_groups_ids = np.array((events_new.id))
#austin_groups_ids=list(set(list(austin_groups_ids)))

C:\Users\sijop\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [3]:
## Function to fetch RSVP data
def getrsvps(event_id,off_val):
    event = client.GetRsvps(event_id = event_id, offset = off_val)
    result = event.results
    RES = json_normalize(result)
    return RES

In [3]:
temp_event_ids = event_ids[100:]
tracker = list(range(1,40000,1000))

C:\Users\sijop\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [1]:
#Fetch events and extract to csv
rsvps = pd.DataFrame([])
for i in range(len(temp_event_ids)):
    for j in range(10000):
        try:
            if i in tracker:
                print ("------------------------------"+ str(i) + "------------------------") 
            result = getrsvps(temp_event_ids[i], j)
            if len(result) != 0:
                rsvps = rsvps.append(result)
            else:
                break
        except:
            continue
rsvps.to_csv('rsvps_new_100_rest_all.csv',index=False,encoding='utf-8')

In [2]:
## Store all the RSVP data 
rsvps_new = pd.read_csv("rsvps_new.csv")
rsvps_new_100_rest_all = pd.read_csv('rsvps_new_100_rest_all.csv')

rsvps_all_new = pd.concat([rsvps_new,rsvps_new_100_rest_all])
rsvps_all_new.to_csv('rsvps_all_new.csv',index=False,encoding='utf-8')